# How to use LangChain and Azure OpenAI with Python


Langchain is an open source framework for developing applications using large language models (LLM). <br>

This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [36]:
#%pip install --upgrade openai
#%pip install langchain

## API Configuation and Deployed Model Setup

After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment. 


In [42]:
import openai
import json
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain


# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base=config_details['OPENAI_API_BASE']
    
# API version e.g. "2023-07-01-preview"
openai_api_version=config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-35-turbo-0613"
deployment_name=config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# This is set to `azure`
openai_api_type="azure"

## Deployed Model Setup

In [43]:
# Create an instance of chat llm
llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version=openai_api_version,
    deployment_name=deployment_name,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)

llm([HumanMessage(content="Write me a poem")])

AIMessage(content="In the realm of dreams, where thoughts take flight,\nA tapestry of words, I shall now write.\nWith ink and quill, I'll weave a tale,\nOf love and hope, where hearts prevail.\n\nIn meadows adorned with flowers so fair,\nA gentle breeze whispers secrets in the air.\nThe sun shines bright, painting the sky,\nA canvas of colors, where dreams never die.\n\nBeneath a canopy of stars, we shall dance,\nLost in a moment, in a lover's trance.\nOur hearts entwined, beats synchronized,\nA symphony of love, never compromised.\n\nThrough valleys of sorrow, we shall tread,\nWith courage and strength, our fears we'll shed.\nFor love, a beacon, shall guide our way,\nThrough darkest nights, to a brighter day.\n\nIn the depths of silence, a whispered prayer,\nFor peace and harmony, beyond compare.\nMay kindness bloom, like flowers in spring,\nAnd compassion's song, forever sing.\n\nOh, let this poem be a gentle reminder,\nThat within us all, love is a powerful binder.\nFor in these wor

## PromptTemplates

Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [44]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


# First Example
template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
input_variables=["product_type", "customer_request"],
template=template,
)

print("Example #1:")
print(llm([HumanMessage(content=prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    ))]
))
print("\n")

# Second Example
system_message = "You are an AI assistant travel assistant that provides vacation recommendations."

system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(f"Where should I go on vaction in Decemember for warm weather and beaches?")
print("Example #2:")
print(result)

Example #1:
content='A highly recommended face wash for acne-prone skin is the "Neutrogena Oil-Free Acne Wash." This product contains salicylic acid, which helps to treat and prevent acne by unclogging pores and reducing inflammation. It is oil-free, non-comedogenic, and gentle enough for daily use. Additionally, it effectively removes dirt, excess oil, and makeup without over-drying the skin.' additional_kwargs={} example=False


Example #2:
If you're looking for warm weather and beautiful beaches in December, here are a few destinations you might consider:

1. Maldives: This tropical paradise offers pristine beaches, crystal-clear waters, and luxurious resorts. December is a great time to visit, with temperatures averaging around 28°C (82°F).

2. Thailand: Thailand's southern islands, such as Phuket, Krabi, and Koh Samui, offer warm weather and stunning beaches in December. You can relax on the white sands, go snorkeling or diving, and explore the vibrant local culture.

3. Bali, Ind

## Chains
There are many applications of chains that allow you to combine numerous LLM calls and actions.  <br>

### Simple Sequential Chains  <br>
Allow you to feed the output of one LLM Chain as input for another.

In [45]:
from langchain.chains import SimpleSequentialChain

In [46]:
description_template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
description_prompt_template = PromptTemplate(input_variables=["child_description"], template=description_template)

description_chain = LLMChain(llm=llm, prompt=description_prompt_template)

In [47]:
diy_description_template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
diy_prompt_template = PromptTemplate(input_variables=["diy_project"], template=diy_description_template)

diy_chain = LLMChain(llm=llm, prompt=diy_prompt_template)

In [48]:
overall_chain = SimpleSequentialChain(chains=[description_chain, diy_chain], verbose=True)

In [49]:
review = overall_chain.run("5-year-old girl")



> Entering new SimpleSequentialChain chain...
DIY Sparkling Fairy Wand:

Materials needed:
- Wooden dowel or stick
- Glitter foam sheets in various colors
- Ribbon or tulle
- Craft glue
- Scissors
- Decorative gems or sequins
- Glitter glue (optional)

Instructions:
1. Begin by cutting out a star shape from one of the glitter foam sheets. This will be the top of the wand.
2. Cut out a long strip of foam from another color and wrap it around the wooden dowel or stick, starting from the bottom. Secure it with craft glue.
3. Cut out smaller shapes like hearts, butterflies, or flowers from different colored glitter foam sheets.
4. Use craft glue to stick these shapes onto the wrapped foam strip, creating a beautiful pattern. Let them dry completely.
5. Once the foam shapes are secure, add some extra sparkle by applying glitter glue to the edges or adding decorative gems or sequins.
6. Finally, tie ribbons or tulle strands to the bottom of the wooden dowel or stick for an extra touch of m